In [66]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/powerlifting-base/openipf-2023-10-28-91d2cada.csv


In [67]:
df = pd.read_csv('/kaggle/input/powerlifting-base/openipf-2023-10-28-91d2cada.csv', index_col='Division')

/tmp/ipykernel_32/539111967.py:1: DtypeWarning: Columns (33,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/kaggle/input/powerlifting-base/openipf-2023-10-28-91d2cada.csv', index_col='Division')


what we need:
* name
* sex
* event
* equipment
* age class
* weightclasskg
* tested
* all squat, bench, deadlift, total, place
* country
* parent federation


In [68]:
pd.set_option('display.max_columns', 40) #all of our columns
pd.set_option('display.max_rows', 10)

**I drop unnecessary columns to make the data clearer for further operations.**

In [69]:
columns_to_drop = ['Age','AgeClass','Dots','Wilks','Glossbrenner','Goodlift','State','Date','MeetCountry','MeetState','MeetTown','MeetName']
df = df.drop(columns=columns_to_drop)

**I will quickly discuss the principles of powerlifting. To obtain the overall result, we need to complete 1 of 3 attempts in each exercise. If we fail all attempts in any exercise, we are eliminated from the total score. Therefore, we exclude from our database the results in which the players did not obtain the total result because we will need it. The 4th attempt remains in the database only because it is in the rules, but it is extremely rarely granted to a lifter**

In [70]:
positiv_total_result = (df[['Squat1Kg', 'Squat2Kg', 'Squat3Kg']] > 0).any(axis=1) & (df[['Bench1Kg', 'Bench2Kg', 'Bench3Kg']] > 0).any(axis=1) & (df[['Deadlift1Kg', 'Deadlift2Kg', 'Deadlift3Kg']] > 0).any(axis=1)
df = df[positiv_total_result]

**Now I was sorting records for athletes who compete in the IPF and competed without equipment**

*IPF - International Powerlifting Federation*

In [71]:
is_contest_in_ipf = (df['ParentFederation'] == 'IPF') & (df['Equipment'] == 'Raw') & (df['Sex'] == 'F')
df = df[is_contest_in_ipf]

43 47 kg, 52 kg, 57 kg, 63 kg, 69 kg, 76kg, 84 kg, 84 kg +

In [80]:
weight_classes_female_ipf_current = ['43','47','52','57','63','69','76','84','84+']
weight_classes_female_ipf_only = df['WeightClassKg'].isin(weight_classes_female_ipf_current)
df[weight_classes_female_ipf_only]

,Name,Sex,Event,Equipment,BirthYearClass,BodyweightKg,WeightClassKg,Squat1Kg,Squat2Kg,Squat3Kg,Squat4Kg,Best3SquatKg,Bench1Kg,Bench2Kg,Bench3Kg,Bench4Kg,Best3BenchKg,Deadlift1Kg,Deadlift2Kg,Deadlift3Kg,Deadlift4Kg,Best3DeadliftKg,TotalKg,Place,Tested,Country,Federation,ParentFederation
Division,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Open,Ali Gascoine,F,SBD,Raw,NaN,50.60,52,85.0,90.0,92.5,NaN,92.5,57.5,60.0,62.5,NaN,62.5,137.5,142.5,150.0,NaN,150.0,305.0,2,Yes,New Zealand,OceaniaPF,IPF
Open,Megan Hinchley,F,SBD,Raw,24-39,51.00,52,-90.0,-90.0,90.0,NaN,90.0,50.0,52.5,-55.0,NaN,52.5,100.0,107.5,112.5,NaN,112.5,255.0,5,Yes,Australia,OceaniaPF,IPF
Open,Mary Macken,F,SBD,Raw,50-59,51.20,52,115.0,117.5,120.0,NaN,120.0,55.0,57.5,60.0,NaN,60.0,145.0,147.5,150.0,NaN,150.0,330.0,1,Yes,Australia,OceaniaPF,IPF
Juniors,Rae-Leigh Lyons,F,SBD,Raw,19-23,51.50,52,70.0,75.0,77.5,NaN,77.5,47.5,50.0,52.5,NaN,52.5,102.5,110.0,-117.5,NaN,110.0,240.0,1,Yes,Australia,OceaniaPF,IPF
Open,Elizabeth Craven,F,SBD,Raw,24-39,51.80,52,-92.5,92.5,-95.0,NaN,92.5,60.0,62.5,66.0,NaN,66.0,117.5,122.5,125.0,NaN,125.0,283.5,3,Yes,Australia,OceaniaPF,IPF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Juniors,Mahailya Reeves,F,SBD,Raw,19-23,141.00,84+,220.0,225.0,230.0,NaN,230.0,152.0,-164.5,164.5,NaN,164.5,190.0,200.0,-222.5,NaN,200.0,594.5,1,Yes,USA,IPF,IPF
Juniors,Regina Meier,F,SBD,Raw,19-23,125.00,84+,175.0,187.5,192.5,NaN,192.5,95.0,102.5,107.5,NaN,107.5,147.5,160.0,165.0,NaN,165.0,465.0,2,Yes,Kazakhstan,IPF,IPF
Juniors,Leah Gregory,F,SBD,Raw,19-23,97.10,84+,165.0,175.0,182.5,NaN,182.5,92.5,100.0,-107.5,NaN,100.0,155.0,165.0,170.0,NaN,170.0,452.5,3,Yes,Canada,IPF,IPF


In [73]:
ipf_weight_class_tuple = ['43','47', '52', '72', '84']
filt_female_weight_class_ipf = df['WeightClass'].isin(ipf_weight_class_tuple)
selected_weight_class_cat = df[filt_female_weight_class_ipf]
weight_class_cat = selected_weight_class_cat.groupby([ 'Sex', 'Country'])
ipf_weight_class_cat = weight_class_cat.get_group(('F','Poland'))
ipf_weight_class_cat

KeyError: 'WeightClass'

In [ ]:
ipf_weight_class_cat['Best3SquatKg'].median()

In [ ]:
male_under_23_yo = poland_country_lifters['Age'] <= 23
lifters_greater_than_median_in_best_sq_male_under_23_yo = poland_country_lifters['Last'][male_under_23_yo].unique()
len(lifters_greater_than_median_in_best_sq_male_under_23_yo)

In [ ]:
filt2_f_under_median = (poland_country_lifters['Best3SquatKg'] < 192.5) & (poland_country_lifters['Sex'] == 'F')
filt_2_f_sum = filt2_f.sum()

In [ ]:
filt2_f_under_median_test = poland_country_lifters['Last'][filt2_f_under_median].value_counts()
filt2_f_under_median_test

In [ ]:
df_chart = lifters_greater_than_median_in_best_sq_female

In [ ]:
plt.pie([filt_2_f_sum, filt2_f_under_median.sum() - filt_2_f_sum], labels=['Results above median', 'Results below median'], autopct='%1.1f%%', startangle=90)
plt.title('Summary results of the best squat for Women relative to the median results of Polish athletes')